In [1]:
import gensim
import pandas as pd
import gensim.downloader as api
import pymorphy2
import tensorflow as tf
morph = pymorphy2.MorphAnalyzer()
ttws = tf.keras.preprocessing.text.text_to_word_sequence
import numpy as np
from sklearn.model_selection import train_test_split


In [2]:
word2vec_model300 = fasttext_model300 = api.load('word2vec-ruscorpora-300')


In [3]:
# word2vec_model300.index2word
def wordToVec(text):
    try:
        m = morph.parse(text)[0]
        wordIndex = "{word}_{pos}".format(word=m.word, pos=m.normalized.tag.POS)
        return word2vec_model300.word_vec(wordIndex)
    except:
        return []
    

In [4]:
sequence_len = 20

def sequence2vec(text):
    spl = ttws(text , filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', lower=True, split=' ')
    res = np.zeros((sequence_len, 300))
    i = 0
    for w in spl:
        if i > sequence_len:
            break
        v = wordToVec(w)
        if len(v) > 0:
            res[i] = np.array([v])
            i += 1
    return res

sequence2vec("привет медвед")

array([[ 0.03231752, -0.00848818, -0.02009445, ..., -0.02186396,
         0.06659944,  0.02148311],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [31]:
df = pd.read_csv("input.txt", sep="\t", encoding="utf-8")
df.head()

,text,text1,internet,modem,ip_phone,videvision,hello,cabel_tv,angry,lk,modile_1,monile_1,tarif,LK_2
0,"У меня проблемы с интернетом, ничего не грузит...",0,1,1,0,0,0,0,0,0,0,0,0,0
1,"ютуб не грузит, лампочка горит",0,1,1,1,0,0,0,0,0,0,0,0,0
2,"привет, у вас есть айпи-телефония?",0,0,0,0,1,0,1,0,0,0,0,0,0
3,вы камеры ставите? я бы хотел поставить себе к...,0,0,0,0,0,1,0,0,0,0,0,0,0
4,"алло, что с инетрнетом? постоянные сбои!",0,1,1,0,0,0,0,0,0,0,0,0,0


In [32]:
np.empty((0,20, 300))

array([], shape=(0, 20, 300), dtype=float64)

In [33]:
df = pd.read_csv("input.txt", sep="\t", encoding="utf-8")
values = df['text'].values
X = np.zeros((len(values),sequence_len, 300))

labels = df.columns
i = 0
for s in values:
    X[i] = sequence2vec(s)
    i += 1
    
del df['text']

y = np.zeros((len(values),len(df.columns)))
for i, yy in df.iterrows():
    y[i]=yy.values

    
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [34]:
print(X_train.shape)
print(y_train.shape)
print(X_train[0])

(20, 20, 300)
(20, 13)
[[ 0.00281422 -0.0652354   0.04814361 ...  0.01992613 -0.02836564
   0.03485549]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 ...
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]]


In [53]:
import autokeras as ak

inputs = ak.Input()
output = ak.DenseBlock()(inputs)
output = ak.ClassificationHead(multi_label=True)(output)

auto_model = ak.AutoModel(inputs=[inputs], 
                          overwrite=True,
                          outputs=[output],
                          max_trials=20)

auto_model.fit(X_train, y_train, epochs=2, batch_size=10)


Epoch 1/2
1/1 [==============================] - 0s 114ms/step - loss: 0.6938 - accuracy: 0.2000 - val_loss: 0.6924 - val_accuracy: 0.1000
Epoch 2/2
1/1 [==============================] - ETA: 0s - loss: 0.6922 - accuracy: 0.30 - 0s 54ms/step - loss: 0.6922 - accuracy: 0.3000 - val_loss: 0.6917 - val_accuracy: 0.1000


Epoch 1/2
1/1 [==============================] - 0s 114ms/step - loss: 0.6930 - accuracy: 0.1000 - val_loss: 0.6916 - val_accuracy: 0.0000e+00
Epoch 2/2
1/1 [==============================] - ETA: 0s - loss: 0.6906 - accuracy: 0.30 - 0s 56ms/step - loss: 0.6906 - accuracy: 0.3000 - val_loss: 0.6902 - val_accuracy: 0.1000


Epoch 1/2
1/1 [==============================] - 0s 134ms/step - loss: 0.6935 - accuracy: 0.0000e+00 - val_loss: 0.6918 - val_accuracy: 0.0000e+00
Epoch 2/2
1/1 [==============================] - ETA: 0s - loss: 0.6915 - accuracy: 0.20 - 0s 52ms/step - loss: 0.6915 - accuracy: 0.2000 - val_loss: 0.6905 - val_accuracy: 0.0000e+00


Epoch 1/2
1/1 [==============================] - 0s 102ms/step - loss: 0.6927 - accuracy: 0.1000 - val_loss: 0.6931 - val_accuracy: 0.0000e+00
Epoch 2/2
1/1 [==============================] - ETA: 0s - loss: 0.6926 - accuracy: 0.20 - 0s 39ms/step - loss: 0.6926 - accuracy: 0.2000 - val_loss: 0.6931 - val_accuracy: 0.0000e+00


Epoch 1/2
1/1 [==============================] - 0s 111ms/step - loss: 0.6930 - accuracy: 0.0000e+00 - val_loss: 0.6928 - val_accuracy: 0.3000
Epoch 2/2
1/1 [==============================] - ETA: 0s - loss: 0.6910 - accuracy: 0.10 - 0s 56ms/step - loss: 0.6910 - accuracy: 0.1000 - val_loss: 0.6919 - val_accuracy: 0.0000e+00


...

Epoch 1/2
1/1 [==============================] - 0s 124ms/step - loss: 0.6922 - accuracy: 0.0000e+00 - val_loss: 0.6909 - val_accuracy: 0.0000e+00
Epoch 2/2
1/1 [==============================] - ETA: 0s - loss: 0.6889 - accuracy: 0.10 - 0s 50ms/step - loss: 0.6889 - accuracy: 0.1000 - val_loss: 0.6891 - val_accuracy: 0.0000e+00


Epoch 1/2
1/1 [==============================] - 0s 124ms/step - loss: 0.6933 - accuracy: 0.0000e+00 - val_loss: 0.6936 - val_accuracy: 0.1000
Epoch 2/2
1/1 [==============================] - ETA: 0s - loss: 0.6931 - accuracy: 0.10 - 0s 51ms/step - loss: 0.6931 - accuracy: 0.1000 - val_loss: 0.6935 - val_accuracy: 0.1000


Epoch 1/2
1/1 [==============================] - 0s 116ms/step - loss: 0.6938 - accuracy: 0.0000e+00 - val_loss: 0.6936 - val_accuracy: 0.0000e+00
Epoch 2/2
1/1 [==============================] - ETA: 0s - loss: 0.6938 - accuracy: 0.10 - 0s 50ms/step - loss: 0.6938 - accuracy: 0.1000 - val_loss: 0.6936 - val_accuracy: 0.0000e+00


Epoch 1/2
1/1 [==============================] - 0s 443ms/step - loss: 0.6946 - accuracy: 0.0000e+00 - val_loss: 0.6881 - val_accuracy: 0.3000
Epoch 2/2
1/1 [==============================] - ETA: 0s - loss: 0.6869 - accuracy: 0.0000e+ - 0s 418ms/step - loss: 0.6869 - accuracy: 0.0000e+00 - val_loss: 0.6826 - val_accuracy: 0.3000


INFO:tensorflow:Oracle triggered exit
Epoch 1/2
2/2 [==============================] - ETA: 0s - loss: 0.6935 - accuracy: 0.0000e+ - 0s 3ms/step - loss: 0.6012 - accuracy: 0.1000
Epoch 2/2
2/2 [==============================] - ETA: 0s - loss: 0.3974 - accuracy: 0.30 - 0s 3ms/step - loss: 0.3815 - accuracy: 0.3000


In [54]:

# print(auto_model.predict(X_test))
print(auto_model.evaluate(X_test, y_test))



2/2 [==============================] - ETA: 0s - loss: 0.3444 - accuracy: 0.10 - 0s 2ms/step - loss: 0.3342 - accuracy: 0.0909
[0.33415907621383667, 0.09090909361839294]


In [204]:
import autokeras as ak
image_input = ak.ImageInput()
image_output = ak.ImageBlock()(image_input)
text_input = ak.TextInput()
text_output = ak.TextBlock()(text_input)
output = ak.Merge()([image_output, text_output])
classification_output = ak.ClassificationHead()(output)
regression_output = ak.RegressionHead()(output)
ak.AutoModel(
    inputs=[image_input, text_input],
    outputs=[classification_output, regression_output]
)